### 4.1 模型构造

In [18]:
import torch
from torch import nn

### 4.1.1 继承Module类构造模型

In [21]:
# Module是nn中的模型构造类,是所有神经网络的基类,
# __init__(): 创建模型参数
# forward(): 定义前向计算

# MLP类无需定义反向传播,会通过自动求梯度而产生反向传输所需的backward()

class MLP(nn.Module):
    # 声明带有模型参数的层,这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数进行必要的初始化。这样在构造实例时还可以指定其他函数
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10) # 输出层
        
    # 前向传播计算
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [22]:
# net(X) 会调用MPL类继承Module类的__call__函数
# Module该类是一个可供自由组建的部件,他的子类可以是一个层,也可以是一个模型
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0084, -0.0396,  0.0371, -0.0867,  0.0613, -0.0768, -0.1162,  0.0773,
         -0.0443, -0.3399],
        [ 0.0084, -0.0284, -0.0222, -0.1399,  0.0358,  0.0561, -0.0828,  0.0181,
         -0.0431, -0.2706]], grad_fn=<AddmmBackward>)

### 4.1.2 Module的子类

### 4.1.2.1 Sequential类

In [23]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict):
            for key, module in args[0].items():
                self.add_module(key, module) # add_module方法会将module添加到self._modules(一个OrderedDict)
        else: # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
                
    def forward(self, input):
        # self._modules 返回一个OrderedDict,保证按照成员添加时的顺序遍历
        for module in self._modules.values():
            input = module(input)
        return input

In [25]:
net = MySequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0498,  0.4772,  0.0545, -0.1967, -0.2779, -0.0585, -0.0498,  0.0343,
         -0.0815, -0.0711],
        [-0.0405,  0.5435, -0.0699, -0.2748, -0.3094,  0.1799,  0.0745,  0.0243,
         -0.0041,  0.0451]], grad_fn=<AddmmBackward>)

### 4.1.2.2 ModuleList类

In [28]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) # 类似list的append
print(net[-1])
print(net)
# net(torch.zeros(1, 784)) NotImplementedError

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [30]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])
        
class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        self.linears = [nn.Linear(10, 10)]
        
net1 = Module_ModuleList()
net2 = Module_List()

print('net1:')
for p in net1.parameters():
    print(p.size())
    
print('net2:')
for p in net2.parameters():
    print(p)

net1:
torch.Size([10, 10])
torch.Size([10])
net2:


### 4.1.2.3 ModuleDict类

In [36]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU()
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['output']) # 访问
print(net.output)
print(net)

Linear(in_features=256, out_features=10, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 4.1.3 构造复杂的模型

In [52]:
class FancyMLP(nn.Module):
    def __init__(self, **kwagrs):
        super(FancyMLP, self).__init__(**kwagrs)
        
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不训练参数
        self.linear = nn.Linear(20, 20)
    
    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数,以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        # 复用全连接层,等价于两个全连接层共享参数
        x = self.linear(x)
        
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

In [53]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-12.3073, grad_fn=<SumBackward0>)

In [54]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__()
        
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())
        
    def forward(self, x):
        return self.net(x)
    
net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(9.1203, grad_fn=<SumBackward0>)